Let's consider a really simple model without even an infection process,
in order to understand the fundamentals of how we should understand transition flows.

In [ ]:
# If we are running in google colab, pip install the required packages, 
# but do not modify local environments
# This will be present in all subsequent notebooks
try:
  import google.colab
  IN_COLAB = True
  %pip install summerepi2
except:
  IN_COLAB = False

In [ ]:
import pandas as pd
import numpy as np
from typing import Dict

from summer2 import CompartmentalModel
from summer2.parameters import Parameter as param

pd.options.plotting.backend = "plotly"

In [ ]:
def get_single_transition_model(
    parameters: dict
) -> CompartmentalModel:
    """
    Generate an instance of a very simple two-compartment model that allows transition
    from one of the two states to the other through a single flow linking them, 
    population distribution and parameters hard-coded.
    This is close to the simplest "closed" system possible,
    such that we're tracking where our starting population goes to and how many
    are left in their origin compartment.
    
    Returns:
        The summer model object
    """
    compartments = (
        "healthy",
        "diseased",
    )
    analysis_times = (0, 20)
    
    model = CompartmentalModel(
        times=analysis_times,
        compartments=compartments,
        infectious_compartments=[],
    )
    model.set_initial_population(
        distribution={"healthy": 1.}
    )
    model.add_transition_flow(
        "onset", 
        fractional_rate=param("onset_rate"), 
        source="healthy", 
        dest="diseased"
    )
    return model

With this single transition flow, 
people in the source compartment transition to the destination compartment 
at a _per capita_ rate defined by the flow's parameter.
By "_per capita_" we mean that the rate of this flow is calculated as the product of the parameter value assigned to the flow rate
and the size of the population in the source compartment.

With just a single outflow from the source compartment, 
the sojourn time is the reciprocal of the rate of outflow.
(By competing flows, we mean flows that have the same source compartment.)
Where multiple flows share a common source compartment, the sojourn time is the reciprocal of the sum of the outflows from that compartment.

In the example below, we will add a progression flow called "onset" where the average time to developing disease is 0.1 time units.
This can be thought of as the mean time to disease onset for someone in the healthy category as being 10 time units.

In [ ]:
transition_params = {
    "population": 1.,
    "onset_rate": 0.1,
}

transition_model = get_single_transition_model(transition_params)

transition_model.run(parameters=transition_params)
compartment_values = transition_model.get_outputs_df()
compartment_values.plot()

In addition to transition flows, 
_summer_ also supports a range of other 
flows that we may wish to apply to our models.

|Flow name|Source compartment?|Destination compartment?|
|---|---|---|
|Infection density|Yes|Yes|
|Infection frequency flow|Yes|Yes|
|Death flow|Yes|No|
|Universal death flows|Yes|No|
Crude birth flow (docs)
Importation flow (docs)
Replacement birth flow (docs)
Function flow (docs)